In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

import os
import numpy as np
import scipy
import matplotlib.pyplot as plt
import sys; sys.path.insert(0, r"C:\Users\lukas\Dokumente\projects\invert")

from invert import Solver

n_sources = 2

In [64]:
# load .mat file
gain = scipy.io.loadmat(r"C:\Users\lukas\Dokumente\projects\flex_ssm\ISBI_Fixed_Oriented_Dipoles_SSM\headmodel_surf_os_meg_01_orig_constrained.mat")
fixed = scipy.io.loadmat(r"C:\Users\lukas\Dokumente\projects\flex_ssm\ISBI_Fixed_Oriented_Dipoles_SSM\FIXED_MEG_DATA.mat")
pos = fixed["Cortex"][0][0][1]
leadfield = gain["Gain"]
leadfield /= leadfield.std(axis=0)


# gain = scipy.io.loadmat(r"C:\Users\lukas\Dokumente\projects\flex_ssm\ISBI_Fixed_Oriented_Dipoles_SSM\savedata\gain.mat")
# leadfield = gain["Gain"]
# pos = gain["Cortex"][0][0][1]
# adjacency = gain["Cortex"][0][0][3]

In [70]:
from scipy.spatial.distance import cdist
from scipy.sparse import csr_matrix
solver_names = ["SSM", "AP", "RAP"]
solvers = {
    "SSM": Solver("SSM", scale_leadfield=False, prep_leadfield=False),
    "AP": Solver("AP", scale_leadfield=False, prep_leadfield=False),
    "RAP": Solver("RAP-MUSIC", scale_leadfield=False, prep_leadfield=False),
}

files = r"C:\Users\lukas\Dokumente\projects\flex_ssm\ISBI_Fixed_Oriented_Dipoles_SSM\savedata"

data_files = [os.path.join(files,f) for f in os.listdir(files) if f.startswith("Y_")]
mean_localization_errors = {solver_name: [] for solver_name in solver_names}
for solver_name in solver_names:
    solver = solvers[solver_name]
    for i, data_file in enumerate(data_files[::19]):
        print(f"{i}/{len(data_files)}")
        eeg = scipy.io.loadmat(data_file)
        Y = eeg['Y']
        solver.leadfields = [leadfield,]
        solver.leadfield = leadfield
        solver.gradients = [csr_matrix(np.identity(leadfield.shape[1])), ]
        # try:
        if solver_name == "SSM":
            solver.make_ssm(Y, n_sources, n_sources, refine_solution=True, max_iter=5,)
        elif solver_name == "AP":
            solver.make_ap(Y, n_sources, n_sources, refine_solution=True, max_iter=6,)
        elif solver_name == "RAP":
            solver.make_flex(Y, n_sources, n_sources, 0, False, refine_solution=False, max_iter=1000,)
        else:
            AttributeError("Solver not found")
            
        idc_est = np.array(sorted([x[1] for x in solver.candidates]))
        idc_true = eeg["curr_pos"][:, 0].astype(int) - 1
        
        distmat = cdist(pos[idc_true, :], pos[idc_est, :])
        MLE = 1000 * (np.mean( np.min(distmat, axis=0) ) +  np.mean( np.min(distmat, axis=1) ) ) / 2  # in mm

        mean_localization_errors[solver_name].append(MLE)
        print(f"{solver_name}: {MLE:.1f} mm")  

        # # 3d scatter plot
        # import matplotlib.pyplot as plt
        # from mpl_toolkits.mplot3d import Axes3D
        
        # fig = plt.figure()
        # ax = fig.add_subplot(111, projection='3d')
        # for point in pos[::100]:
        #     ax.scatter(*point, marker="o", color="grey", s=5)
        # for idx in idc_est:
        #     ax.scatter(*pos[idx, :], marker="o", color="red", s=150)
        # for idx in idc_true:
        #     ax.scatter(*pos[idx, :], marker="o", color="green", s=150)
        # ax.set_title(f"Sample {i}, solver {solver_name}")


        # break
    # break

0/300
SSM: 0.0 mm
1/300
SSM: 0.0 mm
2/300
SSM: 0.0 mm
3/300
SSM: 0.0 mm
4/300
SSM: 0.0 mm
5/300
SSM: 0.0 mm
6/300
SSM: 0.0 mm
7/300
SSM: 0.0 mm
8/300
SSM: 0.0 mm
9/300
SSM: 0.0 mm
10/300
SSM: 0.0 mm
11/300
SSM: 0.0 mm
12/300
SSM: 0.0 mm
13/300
SSM: 0.0 mm
14/300
SSM: 0.0 mm
15/300
SSM: 0.0 mm
0/300
AP: 0.0 mm
1/300
AP: 0.0 mm
2/300
AP: 0.0 mm
3/300
AP: 34.0 mm
4/300
AP: 0.0 mm
5/300
AP: 0.0 mm
6/300
AP: 0.0 mm
7/300
AP: 0.0 mm
8/300
AP: 0.0 mm
9/300
AP: 0.0 mm
10/300
AP: 0.0 mm
11/300
AP: 0.0 mm
12/300
AP: 0.0 mm
13/300
AP: 0.0 mm
14/300
AP: 0.0 mm
15/300
AP: 29.4 mm
0/300
Source Iteration  0
Source Iteration  1
[[0, 5075], [0, 12079]]
RAP: 0.0 mm
1/300
Source Iteration  0
Source Iteration  1
[[0, 14426], [0, 11050]]
RAP: 0.0 mm
2/300
Source Iteration  0
Source Iteration  1
[[0, 13174], [0, 11912]]
RAP: 25.8 mm
3/300
Source Iteration  0
Source Iteration  1
[[0, 6691], [0, 3674]]
RAP: 29.9 mm
4/300
Source Iteration  0
Source Iteration  1
[[0, 12921], [0, 29]]
RAP: 6.1 mm
5/300
Source It

In [71]:
import seaborn as sns
import pandas as pd

df = pd.DataFrame(mean_localization_errors)
sns.barplot(data=df)

<Axes: >

In [73]:
Y.shape

(306, 50)

In [72]:
df.describe()

,SSM,AP,RAP
count,16.0,16.000000,16.000000
mean,0.0,3.964732,9.157094
std,0.0,10.866607,15.358591
min,0.0,0.000000,0.000000
25%,0.0,0.000000,0.000000
50%,0.0,0.000000,0.000000
75%,0.0,0.000000,12.906478
max,0.0,34.031724,50.745076
